In [1]:
import time
import praw
from datetime import datetime
import threading
import pandas as pd

In [2]:
reddit = praw.Reddit(client_id='P26uUpjfnflDVg', client_secret="L9oHqikaIVKSm-aXv-a7E-x8Yi8",
                     password='username', user_agent='script by /u/CandlesTaken',
                     username='password')

In [3]:
class Post:
    commentforrest = reddit.submission('3hahrw').comments
    comment = []
    upvote_count = 0
    def __init__(self, subreddit, title, created_time,Pid):
        self.subreddit = subreddit
        self.title = title
        self.created_time = created_time
        self.Pid = Pid

In [4]:
class Comment:
    def __init__(self, body,Cid):
        self.Cid = Cid
        self.body = body

In [5]:
submissions = []
hots = []

In [17]:
subreddits = [reddit.subreddit('CoronavirusUS'),reddit.subreddit('CoronavirusUK'),reddit.subreddit('CoronavirusNewYork'),reddit.subreddit('CoronavirusCA'),reddit.subreddit('CoronavirusMichigan'),reddit.subreddit('CoronavirusPA'),reddit.subreddit('CoronavirusIllinois')]

In [21]:
for i in subreddits:
    print(i.title)

CoronavirusUS
CoronavirusUK
CoronavirusNewYork
CoronavirusCA
Talk about Coronavirus in Michigan 
CoronaVirusPA
CoronavirusIllinois
CoronaVirusFlorida
CoronavirusIndiana


In [7]:
def submissionstream(subreddits,submissions):#stream#
    string = str("")
    for i in range(0,len(subreddits)):
        if (i != 0):
            string += "+"+str(subreddits[i].title)
        else:
            string += str(subreddits[i].title)
    for submission in reddit.subreddit(string).stream.submissions():
        print(str(submission.subreddit.title) + "; " + submission.title+"; "+str(datetime.utcfromtimestamp(submission.created_utc))+";"+str(submission.score))
        dt = datetime.utcfromtimestamp(submission.created_utc)
        post = Post(submission.subreddit.title, submission.title, dt, submission.id)
        submissions.append(post)
    return submissions

In [8]:
def submissionsupdate(subreddits,submissions):#called at the end of the day#
    for submission in submissions:
        post = reddit.submission(id=submission.Pid)
        submission.upvote_count = post.score
        submission.commentforrest = post.comments
        for comment in submission.commentforrest:
            submission.comment.append(Comment(comment.body, comment.id))
    print("update_done!")
    return submissions

In [9]:
def hot(subreddits,hots):
    for sub in subreddits:
        hotposts = sub.hot(limit = 50)
        for submission in hotposts:
            dt = datetime.utcfromtimestamp(submission.created_utc)
            post = Post(submission.subreddit.title, submission.title, dt, submission.id)
            post.upvote_count = submission.score
            hots.append(post)
        print(sub.title+" done!")
    return hots, datetime.utcnow()

In [10]:
def csvwriter(submissions,hots):
    stream_dic = {"subreddit":[],
                  "Pid":[],
                  "title":[],
                  "upvote":[],
                  "date":[]}
    comment_dic = {"Cid":[],
                   "Pid":[],
                   "body":[]}
    hot_dic = {"subreddit":[],
               "Pid":[],
               "title":[],
               "upvote":[],
               "date":[]}
    for post in submissions:
        stream_dic["subreddit"].append(post.subreddit)
        stream_dic["Pid"].append(post.Pid)
        stream_dic["title"].append(post.title)
        stream_dic["upvote"].append(post.upvote_count)
        stream_dic["date"].append(post.created_time)
        for comment in post.comment:
            comment_dic["Cid"].append(comment.Cid)
            comment_dic["Pid"].append(post.Pid)
            comment_dic["body"].append(comment.body)
    for post in hots:
        hot_dic["subreddit"].append(post.subreddit)
        hot_dic["Pid"].append(post.Pid)
        hot_dic["title"].append(post.title)
        hot_dic["upvote"].append(post.upvote_count)
        hot_dic["date"].append(post.created_time)
    dt = str(datetime.utcnow()).split(" ")[0]
    df_stream = pd.DataFrame(stream_dic)
    df_comment = pd.DataFrame(comment_dic)
    df_hot = pd.DataFrame(hot_dic)
    df_stream.to_csv(str('stream'+dt+'.csv'))
    df_comment.to_csv(str('comment'+dt+'.csv'))
    df_hot.to_csv(str('hot'+dt+'.csv'))

In [22]:
submissionstream(subreddits,submissions)

CoronaVirusFlorida; Coronavirus Outbreak Timelapse January 20 to April 13 Covid 19 Spread Distribution By Continents; 2020-04-12 21:33:44;2
CoronavirusUS; Sidelined by Scandal, a Top Disease Modeler Watches and Worries; 2020-04-12 21:33:53;16
CoronavirusUS; Kentucky church fills Easter service to near capacity, while cops wait in parking lot with 14-day quarantines for all.; 2020-04-12 21:36:20;118
CoronavirusUS; Florida coronavirus update for Sunday: Over 19,300 reported cases, 452 killed in state; 2020-04-12 21:37:17;9
CoronavirusUS; NJ Withholding Federally Allocated Funds To 4 Counties in District 5; 2020-04-12 21:37:50;5
CoronavirusUS; Canada: Has someone in your area been tested for COVID-19? A new website lets you know; 2020-04-12 21:38:30;7
CoronavirusUS; Coronavirus updates in Louisiana: 20,595 COVID-19 cases in the state; 840 deaths reported; 2020-04-12 21:39:08;20
CoronavirusUS; A North Carolina prison complex has 60 inmates and 23 staff members with coronavirus; 2020-04-12 

RequestException: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Max retries exceeded with url: /r/CoronavirusUS+CoronavirusUK+CoronavirusNewYork+CoronavirusCA+Talk%20about%20Coronavirus%20in%20Michigan%20+CoronaVirusPA+CoronavirusIllinois+CoronaVirusFlorida+CoronavirusIndiana/new?limit=86&raw_json=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000127B6D8E048>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [23]:
submissionsupdate(subreddits,submissions)

update_done!


In [24]:
hot(subreddits,hots)

CoronavirusUS done!
CoronavirusUK done!
CoronavirusNewYork done!
CoronavirusCA done!
Talk about Coronavirus in Michigan  done!
CoronaVirusPA done!
CoronavirusIllinois done!
CoronaVirusFlorida done!
CoronavirusIndiana done!


([<__main__.Post at 0x127b75ef208>,
 datetime.datetime(2020, 4, 13, 6, 50, 12, 469058))

In [25]:
csvwriter(submissions,hots)